In [1]:
# Импорт библиотек

import pandas as pd

from scipy.stats import mannwhitneyu
from scipy.stats import shapiro
from scipy.stats import norm
from scipy.stats import t

from statsmodels.stats import power
from statsmodels.stats.proportion import (
    proportion_effectsize,
    proportions_ztest,
    proportion_confint
)


In [2]:
# Загрузка данных

df = pd.read_csv("data/marketing_AB.csv")
df

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14
...,...,...,...,...,...,...,...
588096,588096,1278437,ad,False,1,Tuesday,23
588097,588097,1327975,ad,False,1,Tuesday,23
588098,588098,1038442,ad,False,3,Tuesday,23
588099,588099,1496395,ad,False,1,Tuesday,23


### Проанализируйте структуру данных и проведите их предобработку:

Исследуйте структуру данных;

Преобразуйте столбцы к необходимым типам данных.

Проверьте данные на наличие пропусков и избавьтесь от них, если нужно.

Проверьте, есть ли пользователи, которые в процессе A/Bтеста попали в обе группы. Если да, исключите пользователей, оказавшихся в обеих группах.

In [3]:
# Исследование структуры данных

# Посмотреть первые 5 строк данных
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


In [4]:
# Посмотреть на структуру данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [5]:
# Посмотреть статистику по числовым столбцам
df.describe()

,Unnamed: 0,user id,total ads,most ads hour
count,588101.000000,5.881010e+05,588101.000000,588101.000000
mean,294050.000000,1.310692e+06,24.820876,14.469061
std,169770.279668,2.022260e+05,43.715181,4.834634
min,0.000000,9.000000e+05,1.000000,0.000000
25%,147025.000000,1.143190e+06,4.000000,11.000000
50%,294050.000000,1.313725e+06,13.000000,14.000000
75%,441075.000000,1.484088e+06,27.000000,18.000000
max,588100.000000,1.654483e+06,2065.000000,23.000000


In [6]:
# Проверить наличие дубликатов
df.duplicated().sum()

np.int64(0)

In [7]:
# Преобразуйте столбцы к необходимым типам данных
# Столбцы most ads day и test group преобразуем в категориальные
# Преобразовать столбец test group в категориальный тип
df["test group"] = df["test group"].astype("category")

# Преобразовать столбец most ads day в категориальный тип
df["most ads day"] = df["most ads day"].astype("category")

# Проверить результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Unnamed: 0     588101 non-null  int64   
 1   user id        588101 non-null  int64   
 2   test group     588101 non-null  category
 3   converted      588101 non-null  bool    
 4   total ads      588101 non-null  int64   
 5   most ads day   588101 non-null  category
 6   most ads hour  588101 non-null  int64   
dtypes: bool(1), category(2), int64(4)
memory usage: 19.6 MB


In [8]:
# Проверьте данные на наличие пропусков и избавьтесь от них, если нужно

# Проверить наличие пропусков
df.isnull().sum()

Unnamed: 0       0
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64

Вывод: пропуски отсутствуют -> преобразование не требуется

In [9]:
# Проверьте, есть ли пользователи, которые в процессе A/Bтеста попали в обе группы.

# Посчитать количество пользователей, которые попали в обе группы
ad_psa_users = df.groupby("user id")["test group"].nunique()
ad_psa_users[ad_psa_users > 1].index

Index([], dtype='int64', name='user id')

Вывод: нет пользователей, которые в процессе A/Bтеста попали в обе группы

### Сделайте первичный анализ результатов A/B-тестирования
Рассчитайте вспомогательные показатели для контрольной и тестовой групп:

- количество посещений сайта;
- суммарное количество совершённых покупок;

Сделайте промежуточные выводы по построенной таблице. Сопоставимо ли количество посещений обоих вариантов посадочной страницы? Можно ли говорить о сбалансированности выборок?

В каждой из групп рассчитайте ключевые показатели:

- конверсия;
- среднее количество рекламы, увиденной пользователем.

Сделайте первичные выводы о результатах A/B-тестирования на основе показателей конверсии и среднего количества увиденной рекламы в каждой из групп. По какому(-им) показателю(-ям), на первый взгляд, вариант А эффективнее варианта B и наоборот?

In [10]:
# Предварительно сгруппируем данные по двум группам - контрольной и тестовой
# Сгруппировать данные по группам и посчитать суммарное количество посещений сайта total_visits, суммарное количество совершенных покупок total_purchases и количество уникальных пользователей total_users
grouped_data = df.groupby('test group', observed=False).agg(
    total_visits=('total ads', 'sum'),  # Суммарное количество посещений сайта (увиденной рекламы)
    total_purchases=('converted', 'sum'),  # Суммарное количество покупок (True = 1, False = 0)
    total_users=('user id', 'nunique')  # Количество уникальных пользователей
)

grouped_data

,total_visits,total_purchases,total_users
test group,,,
ad,14014701,14423,564577
psa,582481,420,23524


Вывод: суммарное количество посещений сайта и количество покупок в группе ad значительно больше, чем в группе psa. Можно сделать вывод, что выборки не сбалансированы. Необходимо дальнейшее исследование

In [11]:
# Рассчитать конверсию для каждой из групп

grouped_data["conversion"] = ((grouped_data["total_purchases"] * 100) / grouped_data["total_users"]).round(2)

# Рассчитать среднее количество рекламы, увиденной пользователем для каждой из групп

grouped_data["average_ads"] = (grouped_data["total_visits"] / grouped_data["total_users"])

# Вывести результат
grouped_data

,total_visits,total_purchases,total_users,conversion,average_ads
test group,,,,,
ad,14014701,14423,564577,2.55,24.823365
psa,582481,420,23524,1.79,24.761138


Вывод: конверсия в группе ad выше, чем в группе psa. Среднее количество рекламы, увиденной пользователем, в группе ad также выше, чем в группе psa. Можно сделать вывод, что группа ad эффективнее группы psa

### Проведите статистический анализ результатов A/B-тестирования
Сформулируйте статистические гипотезы, соответствующие поставленным бизнес-вопросам, и выберите статистический тест для их проверки.

Не забудьте проверить данные на нормальность там, где это необходимо.

С помощью аппарата статистических тестов определите:

- Есть ли статистическая разница между конверсиями в группах А и B?
- Есть ли статистическая разница между средними количествами увиденной рекламы в группах А и B?

Подкрепите результаты статистических тестов, построив 95 % доверительные интервалы для:

- конверсий в каждой из групп;
- разницы конверсий в группах;
- среднего количества увиденной рекламы в группах А и B.

Проинтерпретируйте построенные доверительные интервалы для конверсий и среднего количества увиденной рекламы группах. Есть ли перекрытие между доверительными интервалами для групп? Если есть, то о чём это говорит?

In [12]:

# Посчитаем среднее и медиану для конверсии в группах ad и psa 

daily_data = df.groupby(['most ads day','test group'], observed=False).agg({
    'user id':'count',
    'converted':'sum'
}).reset_index().rename(columns={'user id': 'users_count'})


daily_data['conversion']=((daily_data['converted']*100)/daily_data['users_count']).round(2)

daily_data.sort_values(by='most ads day', ascending=True)
conversion_piv = daily_data.groupby('test group', observed=False)['conversion'].agg(
    ['mean', 'median']
)

conversion_piv

,mean,median
test group,,
ad,2.557143,2.46
psa,1.770000,1.63


In [13]:

# Сформулируем нулевую и альтернативную гипотезы

# H0: конверсии в группах ad и psa равны
# H1: конверсии в группе ad выше, чем в группе psa

# Для проверки гипотезы о равенстве конверсий в группах ad и psa используем Z-тест

# Сбалансируем выборки
p_control = conversion_piv.loc['psa','mean']/100
p_treatment = conversion_piv.loc['ad','mean']/100

effect_size = proportion_effectsize(p_control, p_treatment)

alpha = 0.05
beta = 0.2

size_sample = power.NormalIndPower().solve_power(effect_size=effect_size, alpha=alpha, power=1-beta, ratio=1)

print(f'Размер выборки: {size_sample:.0f}')

# Сформируем выборки
control_sample = df[df['test group'] == 'psa'].sample(n=int(size_sample), random_state=0)
treatment_sample = df[df['test group'] == 'ad'].sample(n=int(size_sample), random_state=0)

# Объединим выборки
sample = pd.concat([control_sample, treatment_sample], axis=0)


data = sample.groupby(['test group'], observed=False).agg({
    'user id':'count',
    'converted':'sum',
    'total ads':'sum'
}).reset_index().rename(columns={'user id': 'users_count'})
data['mean_ads']=data['total ads']/data['users_count']
data['mean_ads']=data['mean_ads'].round(2)

data['conversion']=data['converted']*100/data['users_count']
data['conversion']=data['conversion'].round(2)


count_conversions_ad=data[data['test group']=='ad']['converted'].sum()
count_conversions_psa=data[data['test group']=='psa']['converted'].sum()

total_ads_ad=data[data['test group']=='ad']['total ads'].sum()
total_ads_psa=data[data['test group']=='psa']['total ads'].sum()

total_observations_ad=data[data['test group']=='ad']['users_count'].sum()
total_observations_psa=data[data['test group']=='psa']['users_count'].sum()

count_purchase = [count_conversions_ad, count_conversions_psa]
count_ads=[total_ads_ad, total_ads_psa]  
count_observation = [total_observations_ad, total_observations_psa] 
z_stat, p_value = proportions_ztest(count_purchase, count_observation)

print(f'Z-статистика: {z_stat:.2f}')
print(f'p-value: {p_value:.4f}')

Размер выборки: 5319
Z-статистика: 2.69
p-value: 0.0071


Вывод: p-value < 0.05, следовательно, нулевая гипотеза отвергается. Можно сделать вывод, что конверсии в группах ad и psa различаются

In [14]:
# H0: среднее количество увиденной рекламы в группах ad и psa равны
# H1: среднее количество увиденной рекламы в группах ad и psa различается

# Для проверки гипотезы о равенстве среднего количества увиденной рекламы в группах ad и psa используем критерий Манна-Уитни

# Проверить данные на нормальность

alpha = 0.05

shapiro_result_a = shapiro(data[data['test group']=='ad']['total ads'])
shapiro_result_b = shapiro(data[data['test group']=='psa']['total ads'])

print('p-value для группы ad:', shapiro_result_a.pvalue)
print('p-value для группы psa:', shapiro_result_b.pvalue)


# Проверить гипотезу о равенстве средних с помощью критерия Манна-Уитни

alpha = 0.05

results = mannwhitneyu(
    x=data[data['test group']=='ad']['users_count'].sum(),
    y=data[data['test group']=='psa']['users_count'].sum(),
    alternative='two-sided'
)
print('p-value:', round(results.pvalue, 2))


p-value для группы ad: nan
p-value для группы psa: nan
p-value: 1.0


/var/folders/v8/yvjcp58143j87hsd8xmzy54w0000gn/T/ipykernel_99584/3432359995.py:10: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  shapiro_result_a = shapiro(data[data['test group']=='ad']['total ads'])
/var/folders/v8/yvjcp58143j87hsd8xmzy54w0000gn/T/ipykernel_99584/3432359995.py:11: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  shapiro_result_b = shapiro(data[data['test group']=='psa']['total ads'])


Вывод: p-value > 0.05, следовательно, нулевая гипотеза не отвергается. Можно сделать вывод, что среднее количество увиденной рекламы в группах ad и psa равны

In [15]:
# Построить 95% доверительные интервалы для конверсий в каждой из групп

# Разделим данные на две группы
ad = df[df['test group'] == 'ad']
psa = df[df['test group'] == 'psa']

# Напишем функцию для расчета доверительного интервала для конверсии в группе

def get_conf_interval(x, n):
    alpha=0.05
    z = - norm.ppf(alpha / 2)
    eps = z * (x * (1 - x) / n) ** 0.5
    lower_ = x - eps 
    upper_ = x + eps 
    return lower_, upper_

# Рассчитать доверительные интервалы для конверсий в группах ad и psa

lower_ad, upper_ad = get_conf_interval(ad['converted'].mean(), ad['user id'].count())
lower_psa, upper_psa = get_conf_interval(psa['converted'].mean(), psa['user id'].count())

# Вывод результата

print(f'Доверительный интервал для ad: [{lower_ad*100:.2f}, {upper_ad*100:.2f}]')
print(f'Доверительный интервал для psa: [{lower_psa*100:.2f}, {upper_psa*100:.2f}]')

Доверительный интервал для ad: [2.51, 2.60]
Доверительный интервал для psa: [1.62, 1.95]


Вывод: доверительные интервалы для конверсий в группах ad и psa не пересекаются, конверсия в группах не равна

In [16]:
# Построить 95% доверительные интервалы для разницы конверсий в группах

def get_conf_interval_diff(x1, n1, x2, n2):
    alpha=0.05
    z = - norm.ppf(alpha / 2)
    eps = z * ((x1 * (1 - x1) / n1 + x2 * (1 - x2) / n2) ** 0.5)
    lower_ = (x2 - x1) - eps
    upper_ = (x2 - x1) + eps
    return lower_, upper_


# Расчет доверительного интервала для разницы конверсий в группах
lower_diff, upper_diff = get_conf_interval_diff(
    ad['converted'].mean(),
    ad['user id'].count(),
    psa['converted'].mean(),
    psa['user id'].count()
)

print(f'Доверительный интервал для разницы конверсий в группах: [{lower_diff*100:.2f}, {upper_diff*100:.2f}]')

Доверительный интервал для разницы конверсий в группах: [-0.94, -0.60]


Вывод: довертельный интервал для разницы конверсий отрицателен, следовательно, конверсия в группе ad выше, чем в группе psa

In [17]:
# Построить 95% доверительные интервалы для среднего количества увиденной рекламы в группах ad и psa

def get_conf_interval_mean(x_mean, x_std, n):
    alpha=0.05
    k = n - 1
    t_crit = -t.ppf(alpha/2, k)
    eps = t_crit * x_std/(n ** 0.5) 
    lower_ = x_mean - eps
    upper_ = x_mean + eps
    return lower_, upper_

# Расчет доверительного интервала для среднего количества рекламы, увиденной пользователем в группе ad и в группе psa
lower_ad, upper_ad = get_conf_interval_mean(
    ad['total ads'].mean(),
    ad['total ads'].std(),
    ad['user id'].count()
)
lower_psa, upper_psa = get_conf_interval_mean(
    psa['total ads'].mean(),
    psa['total ads'].std(),
    psa['user id'].count()
)

print(f'Доверительный интервал для ad: [{lower_ad:.2f}, {upper_ad:.2f}]')
print(f'Доверительный интервал для psa: [{lower_psa:.2f}, {upper_psa:.2f}]')


Доверительный интервал для ad: [24.71, 24.94]
Доверительный интервал для psa: [24.21, 25.31]


Вывод: доверительные интервалы для среднего количества увиденной рекламы в группах ad и psa пересекаются, следовательно, среднее количество увиденной рекламы в группах совпадает

Общий вывод: доверительные интервалы подтверждают результаты статистических тестов. Конверсия в группе ad выше, чем в группе psa. Среднее количество увиденной рекламы в группах ad и psa совпадает. Можно сделать вывод, что группа ad эффективнее группы psa 